# Environment setup

In [ ]:
import pandas as pd
import pprint
import os
from langchain_community.document_loaders import DataFrameLoader

In [ ]:
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Filter generated testset

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_dataset = Dataset.load_from_disk("eval_dataset")

# Convert to DataFrame
eval_df_syntethic = eval_dataset.to_pandas()
eval_df_syntethic = eval_df_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]
print(len(eval_df_syntethic))
display(eval_df_syntethic.head())

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_dataset_adjacent_chunks = Dataset.load_from_disk("eval_dataset_adjacent_chunks")

# Convert to DataFrame
eval_df_adjacent_chunks_syntethic = eval_dataset_adjacent_chunks.to_pandas()
eval_df_adjacent_chunks_syntethic = eval_df_adjacent_chunks_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]
print(len(eval_df_adjacent_chunks_syntethic))
display(eval_df_adjacent_chunks_syntethic.head())

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_dataset_random_chunks = Dataset.load_from_disk("eval_dataset_random_chunks")

# Convert to DataFrame
eval_df_random_chunks_syntethic = eval_dataset_random_chunks.to_pandas()
eval_df_random_chunks_syntethic = eval_df_random_chunks_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]
print(len(eval_df_random_chunks_syntethic))
display(eval_df_random_chunks_syntethic.head())

In [ ]:
# REMOVE ROWS WITH NAN CHUNK NUM 
# Remove rows where 'column_name' contains NaN values
eval_df_syntethic = eval_df_syntethic[eval_df_syntethic['chunk_num'].notna()]

# Optionally, reset the index of the new dataframe (to avoid gaps in index after removal)
eval_df_syntethic = eval_df_syntethic.reset_index(drop=True)

# Assuming df is your dataframe
eval_df_syntethic['chunk_num'] = eval_df_syntethic['chunk_num'].apply(lambda x: [int(x)])

# Print the resulting dataframe
print(len(eval_df_syntethic))
display(eval_df_syntethic)

In [ ]:
# Concatenate the DataFrames
df = pd.concat([
    eval_df_random_chunks_syntethic, 
    eval_df_syntethic, 
    eval_df_adjacent_chunks_syntethic
], ignore_index=True)

display(df)

In [ ]:
from langchain.schema import HumanMessage, SystemMessage
import pandas as pd
from datasets import Dataset
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm 

# Initialize the Gemini model
model_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0
)

tqdm.pandas()

def evaluate_pair(row):
    # Costruisci un input strutturato per il modello
    messages = [
    SystemMessage(content="Sei un critico che valuta coppie di domande e risposte per una FAQ di un software gestionale. \
        Le coppie di domande e risposte devono soddisfare i seguenti criteri per essere considerate utili per valutare un chatbot destinato al supporto clienti di un software gestionale: \
        1. **Rilevanza**: Devono affrontare temi rilevanti per gli utenti di un software gestionale. \
        2. **Logicità e utilità**: Devono essere logiche e utili per fornire informazioni chiare e pratiche agli utenti. \
        Valuta la seguente coppia e decidi se è utile per testare un chatbot per il supporto clienti. Fornisci un feedback strutturato e dettagliato seguendo il formato specificato."),
    HumanMessage(content=f"**Domanda:** {row['question']}\n**Risposta:** {row['answer']}\n\nValuta questa coppia di domanda-risposta e fornisci un feedback nel seguente formato:\n\
        [Sì/No] \\ Spiegazione delle risposta:\
        - **Motivazione del perchè la coppia domanda-risposta è considerata utile o meno** \
        - **Motivazione sulla rilevanza**: [Breve spiegazione, se rilevante o non rilevante] \
        - **Motivazione sulla logicità**: [Breve spiegazione, se logica e utile o meno] \founda \
        Indica anche eventuali miglioramenti necessari nella domanda o nella risposta.")
    ]

    # Invia l'input strutturato al modello
    response = model_gemini(messages)
    return response.content[:8]  # Estrai l'output del modello

# Apply the evaluation function to each row
df["feedback"] = df.progress_apply(evaluate_pair, axis=1)

In [ ]:
#Save the results to a CSV file
df.to_csv("filtered_testset_withGemini.csv", index=False)

In [ ]:
# Filter rows where 'Sì' is present in the 'feedback' column
filtered_testset = df[df['feedback'].str.contains('Sì', na=False)]

# Reset the index if desired
filtered_testset = filtered_testset.reset_index(drop=True)
filtered_testset = filtered_testset.drop(columns="feedback")

# Display the filtered dataframe
display(filtered_testset)

In [ ]:
import random
extract = random.randint(0, len(filtered_testset))
pprint.pprint(filtered_testset['question'][extract])
pprint.pprint(filtered_testset['answer'][extract])

# Extract the best possible synthetic testset

First prompt:

Ti fornisco tre coppie di domande e risposte relative al contesto di un software gestionale. \
    Il tuo compito è selezionare la coppia migliore da utilizzare in un test set per valutare un chatbot basato su Retrieval-Augmented Generation (RAG). \
    Considera i seguenti criteri per scegliere la coppia migliore:\
    1. **Autocontenute**: La domanda e la risposta devono essere complete e non richiedere informazioni aggiuntive per essere comprese.\
    2. **Chiarezza**: La domanda deve essere chiara, sensata e fornire un contesto sufficiente per essere interpretata correttamente.\
    3. **Pertinenza**: La risposta deve essere direttamente correlata alla domanda e comprensibile per un utente che utilizza il software gestionale.


In [ ]:
import pandas as pd
import random
import re

def gemini_decision(qas):
    messages = [
    SystemMessage(content="Ti fornisco tre coppie di domande e risposte relative al contesto di un software gestionale. \
    Il tuo compito è selezionare la coppia migliore da utilizzare in un test set per valutare un chatbot basato su Retrieval-Augmented Generation (RAG). \
    Considera i seguenti criteri per scegliere la coppia migliore: \
    1. **Autocontenute**: La domanda e la risposta devono essere complete e non richiedere informazioni aggiuntive per essere comprese. Assicurati che il contesto sia sufficientemente chiaro per capire senza ambiguità di cosa si stia parlando. \
    2. **Chiarezza**: La domanda deve essere formulata in modo chiaro, sensato, con una formulazione diretta ed essere realistica nel contesto del software gesionale. Non devono esserci termini vaghi o ambigui che possano confondere e deve essere plausibile che un utente ponga questa domanda. \
    3. **Pertinenza**: La risposta deve essere direttamente correlata alla domanda e comprensibile per un utente che utilizza il software gestionale. Escludi domande che riguardano visualizzazioni o operazioni generiche senza spiegazioni contestuali chiare. \
    Inoltre, **escludi domande troppo vaghe o che trattano aspetti di visualizzazione senza un contesto chiaro**, come quelle che chiedono genericamente \
    <quali informazioni sono visibili in una tabella> senza specificare il contesto o la funzione richiesta. \
    Evita anche domande che chiedono come visualizzare qualcosa senza spiegare di quale dato o schermata si sta parlando."),
    HumanMessage(content=f"Le tre coppie di domande e risposte sono: \
        1. Domanda: {qas[0][0]} - Risposta: {qas[0][1]}\n \
        2. Domanda: {qas[1][0]} - Risposta: {qas[1][1]}\n \
        3. Domanda: {qas[2][0]} - Risposta: {qas[2][1]}\n \
        Seleziona **solo la domanda e la risposta migliori** in base ai criteri sopra descritti, senza alcun altro commento o spiegazione.\
        Rispondi nel formato: \
        Domanda: [domanda migliore]  \
        Risposta: [risposta migliore]")
    ] 

    response = model_gemini(messages)
    response = response.content
    pprint.pprint(response)
    print("\n")

    # Usa una regex per estrarre la domanda e la risposta dalla risposta di Gemini
    match = re.search(r"Domanda:\s*(.*?)\s*Risposta:\s*(.*)", response, re.DOTALL)

    if match:
        question = match.group(1).strip()
        answer = match.group(2).strip()
        return question, answer

    return None, None

# Step 2: Creare una copia del DataFrame
df_copy = filtered_testset.copy()

# Step 3: Lista per salvare le migliori coppie domanda-risposta
best_qas = []

# Step 4: Iterare ed estrarre gruppi di 3 domande-risposte
while len(df_copy) >= 3:  # Continua solo se ci sono almeno 3 righe
    # Estrai 3 righe casuali
    sample = df_copy.sample(3)
    
    # Rimuovi le righe estratte dal DataFrame originale
    df_copy = df_copy.drop(sample.index)
    
    # Crea una lista di tuple (domanda, risposta)
    qas = list(zip(sample['question'], sample['answer']))

    # Interazione con Gemini per scegliere la migliore coppia
    question, answer = gemini_decision(qas)
    
    # Salva la coppia selezionata se valida
    if question and answer:
        best_qas.append((question, answer))

# Step 5: Creare un nuovo DataFrame con le coppie migliori
best_df = pd.DataFrame(best_qas, columns=['question', 'answer'])

# Step 6: Salvare le coppie migliori in un file CSV
best_df.to_csv('best_qas.csv', index=False)

print("Le migliori coppie domanda-risposta sono state salvate in 'best_qas.csv'.")

In [ ]:
import pandas as pd

# Normalize the 'question' column for comparison, keeping the original 'question' columns intact
filtered_testset['normalized_question'] = filtered_testset['question'].str.strip().str.lower()
best_df['normalized_question'] = best_df['question'].str.strip().str.lower()

# Filter filtered_testset to keep rows with questions present in best_df (based on the normalized question)
filtered_matching_questions = filtered_testset[filtered_testset['normalized_question'].isin(best_df['normalized_question'])]

# Reset index of the filtered DataFrame
filtered_matching_questions = filtered_matching_questions.reset_index(drop=True)

# Drop the temporary normalized question column to retain the original question column
filtered_matching_questions = filtered_matching_questions.drop(columns=['normalized_question'])

# Display the filtered dataframe with original 'question' column
display(filtered_matching_questions)

# Optionally, save the filtered dataframe to a CSV file
filtered_matching_questions.to_csv('filtered_matching_questions.csv', index=False)